In [1]:
from bs4 import BeautifulSoup
from requests import get
import json
import random
import string
import sqlite3
import logging


In [2]:
# no list to check whether already included

db_path = '../Data/test.db'
db_name = 'test'
log_path = '../Logs/test.log'
conn = sqlite3.connect(db_path)
c = conn.cursor()

logger = logging.getLogger()
fhandler = logging.FileHandler(filename=log_path, mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [3]:
def generateNode(length):
    letters_and_digits = string.ascii_letters +  string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    node = "_:znode" + result_str
    return node

review_count = 0
no_annotation = 0
no_rating = 0

for page in range (1,414):
    url = f'https://www.rollingstone.com/movies/movie-reviews/page/{page}/'
    soup = BeautifulSoup(get(url).text, 'lxml')
    movie_container = soup.find_all("a", class_="c-card__wrap")
    for link in movie_container:
        review_link = link.get('href')
        if not "movie-reviews" in review_link:
            pass
        else:
            review_soup = BeautifulSoup(get(review_link).text, 'lxml')
            review_html = review_soup.find(id="pmc-movie-review-snippet")
            if review_html is None:
                no_annotation += 1
            else:
                for item in review_html:
                    jsonExtract = str(item).split('\n')[2].replace(";", "")
                data = json.loads("".join(jsonExtract))

                reviewBody = data['description']
                ratingValue = str(data['reviewRating']['ratingValue'])
                bestRating = str(data['reviewRating']['bestRating'])
                worstRating = str(data['reviewRating']['worstRating'])
                reviewRating = "already included"

                node = generateNode(31)
                c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node,review_link,reviewBody,str(reviewRating),ratingValue,bestRating,worstRating))
                conn.commit()
                review_count += 1

logging.debug("Reviews extracted: " + str(review_count) + " , reviews without Review Annotation: " + str(no_annotation))


_:znode4e62zX13OS6R2dMX0Ny6shtvkie3182


In [ ]:
#     # print(link.get('href'))
#     if review_link is not None:
#         if review_link.endswith('html'):
#             review_soup = BeautifulSoup(get(review_link).text, 'lxml')
#             review_html = review_soup.find("div", itemtype="http://schema.org/Review")
#             # print(review_html)
#             reviewBody = ""
#             if review_html is None:
#                 print("Kein Review Annotation")
#                 no_annotation += 1
#             else:
#                 reviewBody_html = review_html.find_all("div", class_="reviewtext")
#                 reviewRating = review_soup.find("div", itemtype="http://schema.org/Rating")
#                 for item in reviewBody_html:
#                     reviewBody += item.p.text
#



# Snippets

# review_soup = BeautifulSoup(get(review_link).text, 'lxml')
# review_html = review_soup.find("div", itemtype="http://schema.org/Review")

# data = json.loads("".join(review_soup.find_all("script", {"type":"application/ld+json"}).contents))

# reviewBody_html = review_html.find_all("div", class_="reviewtext")
# reviewRating = review_soup.find("div", itemtype="http://schema.org/Rating")

# reviewBody += item.p.text

# worstRating = reviewRating.find("meta", itemprop="worstRating")["content"]
# bestRating = reviewRating.find("meta", itemprop="bestRating")["content"]
# ratingValue = reviewRating.find("meta", itemprop="ratingValue")["content"]

# print('node:' + node)
# print('url: '+ review_link)
# print('reviewBody: ' + reviewBody)
# print('worstRating: ' + worstRating)
# print('bestRating: ' + bestRating)
# print('ratingValue: ' + ratingValue)

# c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node,review_link,reviewBody,str(reviewRating),ratingValue,bestRating,worstRating))
# conn.commit()

# soup = BeautifulSoup(get(url).text, 'html.parser')
# movie_container = soup.find_all("div", class_="entry post clearfix")
# for link in movie_container[0].find_all("a"):
#     review_link = link.get('href')
#     print(link.get('href'))
#     if review_link is not None:
#         if review_link.endswith('html'):
#             if review_link in current_reviews:
#                 pass
#             else:
#                 print(review_link)
#                 manche haben schema.org/Review nicht drin
#                 review_count += 1
#         if review_link.endswith('html') & review_link not in current_reviews:
#             print(review_link)
#         else:
#             pass
#                 #nicht alle Einträge sind vollständig (fehlende Bewertung)
#                 # review_count += 1
#     else:
#         pass